In [1]:
import re
import requests as req
from bs4 import BeautifulSoup as bs

import json
import time
from lxml import etree

In [2]:
url = "https://www.applysquare.com/ranking-cn/?country=us"

In [3]:
content = req.get(url)

In [5]:
ins = bs(content.text, 'html.parser')

In [19]:
colleges = ins.find_all(name='a',attrs={"class":"institute-lite-logo"})

In [20]:
print(len(colleges))

20


In [25]:
colleges[0].get('href')

'/institute-cn/princeton/'

In [27]:
href = colleges[0].get('href')


https://www.applysquare.com/institute-cn/princeton/


In [ ]:
college_page_url = 'https://www.applysquare.com' + href
print(college_page_url)
college_con = req.get(college_page_url)
selector = etree.HTML(college_con.text)

In [74]:
cn = selector.xpath("//a[@class='sp-right']/text()")[0].strip()
print(cn)

普林斯顿大学


In [43]:
en = selector.xpath("//p[@class='ellipsis']/text()")[0].strip()
print(en)

Princeton University


In [2]:
college_json = open('college.json')
college_set = json.load(college_json)

In [80]:
ranks = selector.xpath("//h1[@class='ranking-number pull-left rounded']/text()")
ranks_item = selector.xpath("//h2[@class='ranking-name ellipsis']/text()")

In [81]:
print(ranks)
print(ranks_item)

['1', '7', '11', '6', '17']
['US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院']


In [88]:
rank_dict = dict(zip(ranks_item, ranks))

In [89]:
for c in college_set:
    if not c['name'].find(en):
        print(c['name'])
        c['cn'] = cn
        c.update(rank_dict)
        break
    

Princeton University (Princeton)


In [123]:
c_list = [set(c['name'].split('(')[0].replace('&', ' ').lower().split()) for c in college_set]
c_list

[{'arizona', 'state', 'university'},
 {'northeastern', 'university'},
 {'a', 'college', 'm', 'station', 'texas', 'university,'},
 {'california', 'of', 'southern', 'university'},
 {'-', 'dallas', 'of', 'texas', 'university'},
 {'carolina', 'north', 'raleigh', 'state', 'university,'},
 {'at', 'buffalo', 'new', 'of', 'state', 'university', 'york'},
 {'-', 'arlington', 'of', 'texas', 'university'},
 {'at', 'chicago', 'illinois', 'of', 'university'},
 {'georgia', 'institute', 'of', 'technology'},
 {'illinois', 'institute', 'of', 'technology'},
 {'-', 'austin', 'of', 'texas', 'university'},
 {'at', 'carolina', 'charlotte', 'north', 'of', 'university'},
 {'florida', 'of', 'university'},
 {'carnegie', 'mellon', 'university'},
 {'new', 'university', 'york'},
 {'jose', 'san', 'state', 'university'},
 {'pennsylvania', 'state', 'university'},
 {'college', 'maryland,', 'of', 'park', 'university'},
 {'cities', 'minnesota,', 'of', 'twin', 'university'},
 {'lafayette', 'purdue', 'university', 'west'},

In [126]:
c_list[0].intersection(c_list[1])

{'university'}

In [ ]:
def similar(s1, s2):
    len(s1.intersection(s2))

In [6]:
def extract_info(href):
    college_page_url = 'https://www.applysquare.com' + href
    college_con = req.get(college_page_url)
    selector = etree.HTML(college_con.text)
    cn = selector.xpath("//a[@class='sp-right']/text()")[0].strip()
    en = selector.xpath("//p[@class='ellipsis']/text()")[0].strip().lower().replace(',', ' -')
    print(en)
    ranks = selector.xpath("//h1[@class='ranking-number pull-left rounded']/text()")
    ranks_item = selector.xpath("//h2[@class='ranking-name ellipsis']/text()")
    rank_dict = dict(zip(ranks_item, ranks))
    for c in college_set:
        if c['name'].lower().find(en) > -1:
            
            c['cn'] = cn
            c.update(rank_dict)       
            return False
    print(college_page_url)
    return True

In [117]:
tl = 'https://www.applysquare.com/ranking-cn/?country=us&page=%d'
extract_info(colleges[10].get('href'))

Virginia Polytechnic Institute and State University
{'name': 'Capitol Technology University (CapTechU)', 'webpage': 'https://www.captechu.edu', 'loc': 'Maryland, United States'}


True

In [105]:
for c in college_set:
    if c['name'].find(u) > -1:
        print(c['name'])
       
        break

In [ ]:
tl = 'https://www.applysquare.com/ranking-cn/?country=us&page=%d'

for i in range(1, 12):
    url = tl % i
    print(url)
    content = req.get(url)
    ins = bs(content.text, 'html.parser')
    colleges = ins.find_all(name='a',attrs={"class":"institute-lite-logo"})
    import time
    for c in colleges:
        y = extract_info(c.get('href'))
        if y:
            print('page %d no %d' % (i, colleges.index(c)))
        time.sleep(0.3)

https://www.applysquare.com/ranking-cn/?country=us&page=1
princeton university
harvard university
yale university
stanford university
columbia university
university of chicago
massachusetts institute of technology
duke university
university of pennsylvania
johns hopkins university
california institute of technology
dartmouth college
northwestern university
brown university
washington university in st. louis
cornell university
vanderbilt university
university of notre dame
rice university
university of california - berkeley
https://www.applysquare.com/institute-cn/berkeley/
page 1 no 19
https://www.applysquare.com/ranking-cn/?country=us&page=2
emory university
georgetown university
university of california - los angeles
carnegie mellon university
university of southern california
university of virginia
wake forest university
tufts university
university of michigan
university of north carolina at chapel hill
boston college
https://www.applysquare.com/institute-cn/bc/
page 2 no 10
new yor